In [50]:
#-*- encoding: iso-8859-15 -*-
import numpy as np
from easydict import EasyDict as edict

import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#### Configuration / parameters to set

In [23]:
args = edict()
args.seq_len = 30
args.offset = 4
args.cuda = False
args.batch_size = 1

### Data Processing functions and classes

In [3]:
def prepare_text(textsource):
    text = ''
    with open(textsource) as txtsource:
        for line in txtsource:
            line = line.strip().lower()
            line = line.replace(',', '').replace('.', '')
            text += ' ' + line
    return text
# Chevrons müssen noch weg

In [96]:
def prepare_data(text, seq_len, offset):
    # Get all the unique characters appearing in the text 
    chars = sorted(list(set(text)))
    char_idx = dict((c, i) for i, c in enumerate(chars))
    idx_char = dict((i, c) for i, c in enumerate(chars)) #### das brauchen wir später!!!
    print(len(chars))
    
    # Define training samples by splitting the text
    sentences = []
    next_chars = []
    for i in range(0, len(text) - seq_len, offset):
        sentences.append(text[i: i + seq_len])
        next_chars.append(text[i + seq_len])
    print('nr training samples', len(sentences))
    
    # Generate features and labels using one-hot encoding
    X = np.zeros((len(sentences), seq_len, len(chars)), dtype='f')
    y = np.zeros((len(sentences), len(chars)), dtype='f')

    for i, sentence in enumerate(sentences):
        for j, char in enumerate(sentence):
            X[i, j, char_idx[char]] = 1
        y[i, char_idx[next_chars[i]]] = 1
        
    return X, y

In [74]:
class TextDataset(Dataset):
    ''' A text dataset class which implements the abstract class torch.utils.data.Dataset. '''
    def __init__(self, txt_filepath, seq_len, offset):
        text = prepare_text(txt_filepath)
        self.data, self.target = prepare_data(text, seq_len, offset)
        
    def __getitem__(self, index):
        ''' Get the data for one training sample (by index) '''
        return self.data[index,:,:], self.target[index,:] 
    
    def __len__(self):
        ''' Get the number of training samples '''
        return self.data.shape[0]

### LSTM functions and classes

In [180]:
class LSTM_RNN(nn.Module):
    
    def __init__(self, input_shape = (30, 61)):
        super(LSTM_RNN, self).__init__()
        
        self.lstm = nn.LSTM(input_size = input_shape[0]*input_shape[1], hidden_size = 128)
        self.linear = nn.Linear(in_features = 128, out_features = 61)
        self.relu = nn.ReLu()
        self.softmax = nn.Softmax()
        #self.lstm_rnn = nn.Sequential(
            #nn.LSTM(input_size = input_shape[0]*input_shape[1], hidden_size = 128)
            #nn.Linear(in_features = 128, out_features = 61),
            #nn.ReLu(), 
            #nn.Softmax()
        #)
        
    def forward(self, x):
        lstm_out, _, _ = self.lstm(x)
        res = self.softmax(self.relu(self.linear(lstm_out))) 
        #res = self.lstm_rnn(x)
        return res       

In [193]:
# Training loop (one epoch)
def train(model, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss() # use the cross-entropy loss
    total_loss = 0.0 # compute total loss over one epoch
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(data.size(0), -1)
        if args.cuda:
            data, target = data.cuda(), target.cuda()   
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        print(output)
        loss = criterion(output, data) # check how far away the output is from the original data
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data[0]
    print('Total loss over epoch %s: %s' %(epoch, total_loss/len(train_loader.dataset)))

### Main code

In [194]:
# test the functions defined above
textsource = './Brown_Leseprobe.txt'
text = prepare_text(textsource)
feat, true_pred = prepare_data(text, args.seq_len, args.offset)
print(feat.shape, true_pred.shape)

61
('nr training samples', 8261)
((8261, 30, 61), (8261, 61))


In [195]:
# Generate train and test loader from our data
train_set = TextDataset('./Brown_Leseprobe.txt', args.seq_len, args.offset)
train_loader = DataLoader(train_set, batch_size = args.batch_size, shuffle=True)
dataiter = iter(train_loader)
print(dataiter.next())

### TEST DATA NOCH

61
('nr training samples', 8261)
[
(0 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
[torch.FloatTensor of size 1x30x61]
, 

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     1     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 39 to 51 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 52 to 60 
    0     0     0     0     0     0     0     0     0
[torch.FloatTensor of size 1x61]
]


In [196]:
# Generate model
rnn = LSTM_RNN()
if args.cuda:
    rnn.cuda()

AttributeError: 'module' object has no attribute 'ReLu'

In [197]:
# Initialize the optimization algorithm
optimizer = optim.RMSprop(net.parameters(), lr=0.01)

In [198]:
for epoch in range(1,2):
    train(rnn, epoch)

(Variable containing:
( 0  ,.,.) = 
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
           ...             ⋱             ...          
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
[torch.FloatTensor of size 1x1830x128]
, (Variable containing:
( 0  ,.,.) = 
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
           ...             ⋱             ...          
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
  0.0832 -0.1171 -0.0169  ...  -0.0308 -0.0566  0.1559
[torch.FloatTensor of size 1x1830x128]
, Variable containing:
( 0  ,.,.) = 
  0.1660 -0.2677 -0.0344  ...  -0.0547 -0

RuntimeError: log_softmax(): argument 'input' (position 1) must be Variable, not tuple

In [179]:
m = nn.Softmax()
input = autograd.Variable(torch.randn(2, 3))
print(input)
print(m(input))

Variable containing:
 1.6369  0.2345 -0.5333
 0.4237 -0.0249 -2.7142
[torch.FloatTensor of size 2x3]

Variable containing:
 0.7352  0.1809  0.0839
 0.5946  0.3796  0.0258
[torch.FloatTensor of size 2x3]



/home/immd-user/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.
